In [1]:
import os
import rasterio
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
from rasterio.features import geometry_mask

# -----------------------------------------------------------------------------
# 1) FILE PATHS (for Assam)
# -----------------------------------------------------------------------------
rh_tiff_path        = "Assam_X_Relative_Humidity.tif"         # 20 bands (June–Sept 2020–2024)
precip_raw_tiff     = "Assam_Y_Precipitation_CHIRPS.tif"      # 25 bands (2020–2024 × 5)
precip_cat_tiff     = "Assam_Y_Precipitation_GT_geotif.tif"   # 25 bands (2020–2024 × 5)
india_shapefile     = "SateMask/gadm41_IND_1.shp"            # contains all Indian states

for path in (rh_tiff_path, precip_raw_tiff, precip_cat_tiff, india_shapefile):
    if not os.path.isfile(path):
        raise FileNotFoundError(f"Cannot find {path!r} in {os.getcwd()}")

# -----------------------------------------------------------------------------
# 2) READ & MASK THE 20-BAND RELATIVE HUMIDITY TIFF
# -----------------------------------------------------------------------------
with rasterio.open(rh_tiff_path) as src_rh:
    rh_bands      = src_rh.read().astype(np.float32)  # (20, H, W)
    rh_transform  = src_rh.transform
    rh_crs        = src_rh.crs
    height, width = src_rh.height, src_rh.width
    rh_bounds     = src_rh.bounds

if rh_bands.shape[0] != 20:
    raise ValueError(f"Expected 20 bands in {rh_tiff_path}, but found {rh_bands.shape[0]}")

# 2a) Load Assam polygon and reproject if needed
gdf = gpd.read_file(india_shapefile)
gdf_assam = gdf[gdf["NAME_1"].str.lower() == "assam"].copy()
if gdf_assam.empty:
    raise ValueError("No feature named 'Assam' found in shapefile.")
if gdf_assam.crs != rh_crs:
    gdf_assam = gdf_assam.to_crs(rh_crs)

# 2b) Build mask and count pixels
assam_geom = [gdf_assam.geometry.union_all()]
assam_mask = geometry_mask(
    assam_geom,
    transform=rh_transform,
    invert=True,
    out_shape=(height, width)
)
total_inside = np.count_nonzero(assam_mask)
print(f"Total pixels inside Assam boundary: {total_inside}")

# 2c) Mask RH outside Assam
for i in range(rh_bands.shape[0]):
    band = rh_bands[i]
    band[~assam_mask] = np.nan
    rh_bands[i] = band

# -----------------------------------------------------------------------------
# 3) READ & MASK RAW PRECIPITATION (25 bands)
# -----------------------------------------------------------------------------
with rasterio.open(precip_raw_tiff) as src_pr:
    pr_raw_full = src_pr.read().astype(np.float32)  # (25, H, W)

if pr_raw_full.shape[0] != 25:
    raise ValueError(f"Expected 25 bands in {precip_raw_tiff}, but found {pr_raw_full.shape[0]}")

for i in range(25):
    arr = pr_raw_full[i]
    arr[~assam_mask] = np.nan
    pr_raw_full[i] = arr

# -----------------------------------------------------------------------------
# 4) READ & MASK PRECIPITATION CATEGORY (25 bands)
# -----------------------------------------------------------------------------
with rasterio.open(precip_cat_tiff) as src_pc:
    pr_cat_full = src_pc.read().astype(np.int8)  # (25, H, W)

if pr_cat_full.shape[0] != 25:
    raise ValueError(f"Expected 25 bands in {precip_cat_tiff}, but found {pr_cat_full.shape[0]}")

for i in range(25):
    arr = pr_cat_full[i]
    arr[~assam_mask] = -1
    pr_cat_full[i] = arr

# -----------------------------------------------------------------------------
# 5) EXTRACT 20 MONTHLY BANDS (drop each year’s “Total”)
# -----------------------------------------------------------------------------
monthly_indices = []
for yr in range(5):
    base = yr * 5
    monthly_indices += [base + m for m in range(4)]

pr_raw_bands = pr_raw_full[monthly_indices]  # (20, H, W)
pr_cat_bands = pr_cat_full[monthly_indices]  # (20, H, W)

# -----------------------------------------------------------------------------
# 6) SET UP YEARS & MONTHS
# -----------------------------------------------------------------------------
years       = [2020, 2021, 2022, 2023, 2024]
month_names = ["June", "July", "August", "September"]
n_years, n_months = len(years), len(month_names)
n_total = n_years * n_months  # 20

# -----------------------------------------------------------------------------
# 7) COLORMAPS & STYLE
# -----------------------------------------------------------------------------
rh_palette = ["red","orange","yellow","white","green","cyan","lightblue","blue"]
rh_cmap, rh_vmin, rh_vmax = ListedColormap(rh_palette), 0, 100

cluster_colors = ["#ffffff","#79d151","#22a784","#29788e","#404387","#440154"]
cluster_cmap    = ListedColormap(cluster_colors)
cluster_labels  = {
    0: "NoData/Outside",
    1: "Scarcity (<0.4×normal)",
    2: "Deficit (0.4–0.8×normal)",
    3: "Normal (0.8–1.2×normal)",
    4: "Excess (1.2–1.6×normal)",
    5: "LargeExcess (≥1.6×normal)"
}
cluster_handles = [
    Patch(facecolor=cluster_colors[i], edgecolor="black", label=cluster_labels[i])
    for i in range(len(cluster_colors))
]

# -----------------------------------------------------------------------------
# 8) BUILD FIGURE
# -----------------------------------------------------------------------------
fig, axes = plt.subplots(n_total, 3, figsize=(20, n_total * 2.0), constrained_layout=True)
fig.suptitle("Assam 2020–2024: Monthly RH | Precip Category | RH Distribution", fontsize=18, y=0.98)
if n_total == 1:
    axes = axes[np.newaxis, :]

# -----------------------------------------------------------------------------
# 9) LOOP & PLOT
# -----------------------------------------------------------------------------
for i in range(n_total):
    yr_idx, mo_idx = divmod(i, n_months)
    year, month = years[yr_idx], month_names[mo_idx]

    # RH stats + missing
    rh_layer = rh_bands[i]
    valid_rh = rh_layer[~np.isnan(rh_layer)]
    rh_min, rh_max = (float(np.nanmin(valid_rh)), float(np.nanmax(valid_rh))) if valid_rh.size else (None,None)
    miss_rh = np.count_nonzero(np.isnan(rh_layer[assam_mask]))
    pct_rh = miss_rh / total_inside * 100
    print(f"{year} {month} → Missing RH: {miss_rh}/{total_inside} ({pct_rh:.2f}%)")

    # precip stats
    pr_layer = pr_raw_bands[i]
    valid_pr = pr_layer[~np.isnan(pr_layer)]
    pr_min, pr_max = (float(np.nanmin(valid_pr)), float(np.nanmax(valid_pr))) if valid_pr.size else (None,None)

    # category prep
    cat = pr_cat_bands[i]
    cat_plot = np.zeros_like(cat, dtype=np.int8)
    mask_ok = (cat != -1)
    cat_plot[mask_ok] = cat[mask_ok] + 1

    # --- Column 0: RH Map ---
    ax0 = axes[i, 0]
    im0 = ax0.imshow(rh_layer, cmap=rh_cmap, vmin=rh_vmin, vmax=rh_vmax,
                     extent=[rh_bounds.left, rh_bounds.right, rh_bounds.bottom, rh_bounds.top],
                     origin="upper")
    title0 = f"{year} {month} RH (%)"
    if rh_min is not None:
        title0 += f"\nmin={rh_min:.1f}%, max={rh_max:.1f}%"
    title0 += f"\nmissing={miss_rh}/{total_inside} ({pct_rh:.1f}%)"
    ax0.set_title(title0, fontsize=10, loc="left")
    ax0.axis("off")
    c0 = fig.colorbar(im0, ax=ax0, fraction=0.045, pad=0.02)
    c0.set_label("RH (%)", fontsize=8); c0.ax.tick_params(labelsize=6)

    # --- Column 1: Precip Category Map ---
    ax1 = axes[i, 1]
    im1 = ax1.imshow(cat_plot, cmap=cluster_cmap, vmin=0, vmax=5,
                     extent=[rh_bounds.left, rh_bounds.right, rh_bounds.bottom, rh_bounds.top],
                     origin="upper")
    title1 = f"{year} {month} Precip Cat\nmin={pr_min:.1f} mm, max={pr_max:.1f} mm"
    ax1.set_title(title1, fontsize=10, loc="left")
    ax1.axis("off")
    c1 = fig.colorbar(im1, ax=ax1, fraction=0.045, pad=0.02)
    c1.set_ticks(list(range(6)))
    c1.set_ticklabels([cluster_labels[j] for j in range(6)], fontsize=6)
    c1.ax.tick_params(labelsize=6); c1.set_label("Precip Category", fontsize=8)

    # --- Column 2: RH Distribution ---
    ax2 = axes[i, 2]
    if valid_rh.size:
        ax2.hist(valid_rh.flatten(), bins=30, density=True, edgecolor="black")
        ax2.set_xlim(rh_vmin, rh_vmax)
    else:
        ax2.text(0.5, 0.5, "No Data", ha="center", va="center", fontsize=8, color="gray")
    ax2.set_title(f"RH Dist\n({year} {month})", fontsize=10)
    ax2.set_xlabel("RH (%)", fontsize=8); ax2.set_ylabel("Density", fontsize=8)
    ax2.tick_params(labelsize=7)

# -----------------------------------------------------------------------------
# 10) Legend
# -----------------------------------------------------------------------------
fig.legend(handles=cluster_handles, loc="lower center", ncol=3,
           frameon=True, title="Precip Category Definitions",
           bbox_to_anchor=(0.5, -0.005), fontsize=8, title_fontsize=9)

plt.show()


FileNotFoundError: Cannot find 'Assam_X_Relative_Humidity.tif' in /home/aparajita/Desktop/Weather Analytics/Weather_Analytics/GridData/Assam